**Modulo 5 : Regularización y Dropout**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Parte 5.5: Benchmarking de Técnicas de Regularización

Hasta ahora se han introducido bastantes hiperparámetros. Ajustar cada uno de estos valores puede tener un efecto en la puntuación obtenida por sus redes neuronales. Algunos de los hiperparámetros vistos hasta ahora incluyen:

* Número de capas en la red neuronal
* Cuantas neuronas hay en cada capa
* Qué funciones de activación usar en cada capa
* Porcentaje de abandono en cada capa
* Valores L1 y L2 en cada capa

Para probar cada uno de estos hiperparámetros, deberá ejecutar entrenar redes neuronales con múltiples configuraciones para cada hiperparámetro. Sin embargo, es posible que haya notado que las redes neuronales a menudo producen resultados algo diferentes cuando se entrenan varias veces. Esto se debe a que las redes neuronales comienzan con pesos aleatorios. Debido a esto, es necesario ajustar y evaluar los tiempos de una red neuronal para garantizar que un conjunto de hiperparámetros sea realmente mejor que otro. Bootstrapping puede ser un medio eficaz de evaluación comparativa (comparación) de dos conjuntos de hiperparámetros.

Bootstrapping es similar a la validación cruzada. Ambos pasan por una serie de ciclos/pliegues que proporcionan conjuntos de validación y entrenamiento. Sin embargo, el arranque puede tener un número ilimitado de ciclos. Bootstrapping elige un nuevo tren y la validación divide cada ciclo, con reemplazo. El hecho de que cada ciclo se elija con reemplazo significa que, a diferencia de la validación cruzada, a menudo se seleccionarán filas repetidas entre ciclos. Si ejecuta el programa de arranque durante suficientes ciclos, habrá ciclos duplicados.

En esta parte, utilizaremos bootstrapping para la evaluación comparativa de hiperparámetros. Entrenaremos una red neuronal para un número específico de divisiones (indicado por la constante SPLITS). Para estos ejemplos usamos 100. Compararemos el puntaje promedio al final de los 100. Al final de los ciclos, el puntaje promedio habrá convergido un poco. Esta puntuación final será una base de comparación mucho mejor que una única validación cruzada. Además, se rastreará el número promedio de épocas para dar una idea de un posible valor óptimo. Debido a que el conjunto de validación de detención temprana también se usa para evaluar la red neuronal, podría estar ligeramente inflado. Esto se debe a que nos detenemos y evaluamos en la misma muestra. Sin embargo, estamos usando las puntuaciones solo como medidas relativas para determinar la superioridad de un conjunto de hiperparámetros sobre otro, por lo que esta ligera inflación no debería presentar demasiado problema.

Debido a que estamos evaluando comparativamente, mostraremos la cantidad de tiempo necesario para cada ciclo. La siguiente función se puede usar para formatear bien un lapso de tiempo.

In [1]:
# Cadena de tiempo bien formateada
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

###Bootstrapping para regresión

El bootstrapping de regresión usa el objeto **ShuffleSplit** para realizar las divisiones. Esto es similar a **KFold** para la validación cruzada, no se realiza el balanceo. Para demostrar esta técnica, intentaremos predecir la columna de edad para el conjunto de datos jh-simple. Estos datos se cargan con el siguiente código.

In [2]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Generar dummies para 'product'
df = pd.concat([df,pd.get_dummies(df['product'],prefix="product")],axis=1)
df.drop('product', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('age').drop('id')
x = df[x_columns].values
y = df['age'].values

El siguiente código realiza el arranque. La arquitectura de la red neuronal se puede ajustar para comparar muchas configuraciones diferentes.

In [4]:
import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ShuffleSplit

SPLITS = 50

# Bootstrap
boot = ShuffleSplit(n_splits=SPLITS, test_size=0.1, random_state=42)

# Sigue el progreso
mean_benchmark = []
epochs_needed = []
num = 0

# Bucle a través de muestras
for train, test in boot.split(x):
    start_time = time.time()
    num+=1

    # Dividir en train/test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construir la red neural
    model = Sequential()
    model.add(Dense(20, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=0, mode='auto', restore_best_weights=True)

    # Train en el bootstrap
    model.fit(x_train,y_train,validation_data=(x_test,y_test),
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    
    # Predict en el arranque (validation)
    pred = model.predict(x_test)
  
    # bootstrap's log loss
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    
    # Grabar esta iteración
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}, stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)},time={hms_string(time_took)}")

#1: score=0.598767, mean score=0.598767, stdev=0.000000, epochs=134, mean epochs=134,time=0:00:21.94
#2: score=1.057863, mean score=0.828315, stdev=0.229548, epochs=104, mean epochs=119,time=0:00:15.15
#3: score=0.666891, mean score=0.774507, stdev=0.202284, epochs=167, mean epochs=135,time=0:00:24.05
#4: score=0.625468, mean score=0.737247, stdev=0.186692, epochs=99, mean epochs=126,time=0:00:14.51
#5: score=0.608584, mean score=0.711515, stdev=0.174734, epochs=126, mean epochs=126,time=0:00:18.39
#6: score=0.730423, mean score=0.714666, stdev=0.159665, epochs=93, mean epochs=120,time=0:00:13.64
#7: score=0.517583, mean score=0.686511, stdev=0.163117, epochs=129, mean epochs=121,time=0:00:18.61
#8: score=0.600120, mean score=0.675712, stdev=0.155234, epochs=132, mean epochs=123,time=0:00:19.15
#9: score=0.733291, mean score=0.682110, stdev=0.147470, epochs=121, mean epochs=122,time=0:00:17.50
#10: score=1.041067, mean score=0.718006, stdev=0.176548, epochs=102, mean epochs=120,time=0:

The bootstrapping process for classification is similar and is presented in the next section.

### Bootstrapping for Clasificación

El arranque de regresión usa el objeto **StratifiedShuffleSplit** para realizar las divisiones. Esto es similar a **StratifiedKFold** para la validación cruzada, ya que las clases están equilibradas para que el muestreo no tenga efecto en las proporciones. Para demostrar esta técnica, intentaremos predecir la columna del producto para el conjunto de datos jh-simple. Estos datos se cargan con el siguiente código.

In [5]:
import pandas as pd
from scipy.stats import zscore

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Clasificación
products = dummies.columns
y = dummies.values

In [6]:
import pandas as pd
import os
import numpy as np
import time
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit

SPLITS = 50

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1, 
                                random_state=42)

# Sigue el profreso
mean_benchmark = []
epochs_needed = []
num = 0

# Bucle a través de muestras
for train, test in boot.split(x,df['product']):
    start_time = time.time()
    num+=1

    # Dividir train/test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construir la red neural
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Oculta 1
    model.add(Dense(25, activation='relu')) # Oculta 2
    model.add(Dense(y.shape[1],activation='softmax')) # Salida
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=25, verbose=0, mode='auto', restore_best_weights=True)

    # Train en el bootstrap
    model.fit(x_train,y_train,validation_data=(x_test,y_test),
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    
    # Predict al arranque (validación)
    pred = model.predict(x_test)
  
    # bootstrap's log loss
    y_compare = np.argmax(y_test,axis=1) # para log loss
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    
    # Grabar esta iteración
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}," +\
          f"stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}," +\
          f" time={hms_string(time_took)}")

#1: score=0.660619, mean score=0.660619,stdev=0.000000, epochs=86, mean epochs=86, time=0:00:12.87
#2: score=0.686717, mean score=0.673668,stdev=0.013049, epochs=65, mean epochs=75, time=0:00:09.78
#3: score=0.679140, mean score=0.675492,stdev=0.010962, epochs=47, mean epochs=66, time=0:00:07.34
#4: score=0.683686, mean score=0.677541,stdev=0.010135, epochs=72, mean epochs=67, time=0:00:10.80
#5: score=0.679965, mean score=0.678025,stdev=0.009117, epochs=64, mean epochs=66, time=0:00:09.46
#6: score=0.693619, mean score=0.680624,stdev=0.010150, epochs=53, mean epochs=64, time=0:00:07.85
#7: score=0.709699, mean score=0.684778,stdev=0.013850, epochs=50, mean epochs=62, time=0:00:07.48
#8: score=0.740468, mean score=0.691739,stdev=0.022518, epochs=43, mean epochs=60, time=0:00:06.59
#9: score=0.617937, mean score=0.683539,stdev=0.031443, epochs=66, mean epochs=60, time=0:00:09.92
#10: score=0.651650, mean score=0.680350,stdev=0.031326, epochs=88, mean epochs=63, time=0:00:13.06
#11: scor

### Benchmarking

Ahora que hemos visto cómo arrancar tanto con la clasificación como con la regresión, podemos comenzar a intentar optimizar los hiperparámetros para los datos del conjunto de datos jh-simple. Para este ejemplo, codificaremos para la clasificación de la columna del producto. La evaluación será en pérdida de registro.

In [7]:
import pandas as pd
from scipy.stats import zscore

# Leer el dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],
               axis=1)
df.drop('area', axis=1, inplace=True)

# Tratar Missing values en income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Clasificación
products = dummies.columns
y = dummies.values

Realicé algunas optimizaciones y el código está configurado actualmente con la mejor configuración que se me ocurrió. Más adelante en este curso veremos cómo podemos utilizar un proceso automático para optimizar los hiperparámetros.

In [9]:
import pandas as pd
import os
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU

SPLITS = 100

# Bootstrap
boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)

# Sigue el progreso
mean_benchmark = []
epochs_needed = []
num = 0

# Bucle a través de muestras
for train, test in boot.split(x,df['product']):
    start_time = time.time()
    num+=1

    # Dividir train/test
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]

    # Construir la red neural
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation=PReLU(), \
        kernel_regularizer=regularizers.l2(1e-4))) # Oculta 1
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=PReLU(), \
        activity_regularizer=regularizers.l2(1e-4))) # Oculta 2
    model.add(Dropout(0.5))
    model.add(Dense(100, activation=PReLU(), \
        activity_regularizer=regularizers.l2(1e-4)
    )) # Oculta 3
#    model.add(Dropout(0.5)) - Por lo general, un mejor rendimiento
# sin abandono en la capa final
    model.add(Dense(y.shape[1],activation='softmax')) # Salida
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)

    # Train en el bootstrap
    model.fit(x_train,y_train,validation_data=(x_test,y_test), \
              callbacks=[monitor],verbose=0,epochs=1000)
    epochs = monitor.stopped_epoch
    epochs_needed.append(epochs)
    
    # Predict al arranque (validación)
    pred = model.predict(x_test)
  
    # bootstrap's log loss
    y_compare = np.argmax(y_test,axis=1) # para el cálculo de log loss
    score = metrics.log_loss(y_compare, pred)
    mean_benchmark.append(score)
    m1 = statistics.mean(mean_benchmark)
    m2 = statistics.mean(epochs_needed)
    mdev = statistics.pstdev(mean_benchmark)
    
    # Guardar esta iteración
    time_took = time.time() - start_time
    print(f"#{num}: score={score:.6f}, mean score={m1:.6f}, stdev={mdev:.6f}, epochs={epochs}, mean epochs={int(m2)}, time={hms_string(time_took)}")

#1: score=0.591907, mean score=0.591907, stdev=0.000000, epochs=217, mean epochs=217, time=0:00:56.38
#2: score=0.660223, mean score=0.626065, stdev=0.034158, epochs=190, mean epochs=203, time=0:00:49.85
#3: score=0.696548, mean score=0.649559, stdev=0.043380, epochs=210, mean epochs=205, time=0:00:54.98
#4: score=0.614519, mean score=0.640799, stdev=0.040517, epochs=198, mean epochs=203, time=0:00:51.70
#5: score=0.603057, mean score=0.633251, stdev=0.039258, epochs=194, mean epochs=201, time=0:00:50.77
#6: score=0.774661, mean score=0.656819, stdev=0.063731, epochs=162, mean epochs=195, time=0:00:42.68
#7: score=0.624923, mean score=0.652263, stdev=0.060050, epochs=176, mean epochs=192, time=0:00:46.23
#8: score=0.678533, mean score=0.655546, stdev=0.056840, epochs=213, mean epochs=195, time=0:00:54.34
#9: score=0.630719, mean score=0.652788, stdev=0.054154, epochs=195, mean epochs=195, time=0:01:58.95
#10: score=0.693280, mean score=0.656837, stdev=0.052792, epochs=156, mean epochs=